<a href="https://colab.research.google.com/github/steciow/Colaborador/blob/main/COVID_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROCESAMIENTO DE BASE DIARIA DE CASOS REGISTRADOS EN ARGENTINA**
**Casos COVID-19 en Argentina**

Fuente:

Base diaria de Casos Registrados en la República Argentina de la página de datos abiertos: 
https://datos.gob.ar/dataset/salud-covid-19-casos-registrados-republica-argentina  (luego de las 20 hs ya se encuentra cargada la información del día en curso).
Se va a leer la tabla directamente de ese repositorio y se elaborarán tablas resumen.

## **Importar librerías**

In [ ]:
# importamos de librerías que se utilizarán para trabajar las tablas y le otorgamos un alias
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import plotly.express as px

In [ ]:
#para ver mejor el contenido de las tablas, indico el máximo de filas a visualizar en 400...
#pd.set_option('display.max_rows',800)

## **Levantar bases**

### **Base SISA**

In [ ]:
#levantar desde enlace web
url = 'https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.csv'
dfa = pd.read_csv(url)

###**Base de datos vacunación**

In [ ]:
#levantar desde enlace web
url='https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19VacunasAgrupadas.csv'
dfvacc1= pd.read_csv(url)

### **Bases auxiliares (clasificación y descripciones de distritos)**

In [ ]:
#levantar desde enlace web
url = 'https://raw.githubusercontent.com/steciow/municipios_regiones_PBA/main/info_mun.csv'
dfaux = pd.read_csv(url)

In [ ]:
url='https://raw.githubusercontent.com/steciow/jurisdiccionesARG/main/info_jur.csv'
dfaux2=pd.read_csv(url)

## **Explorando las bases**

In [ ]:
#primera exploración de la base: tamaño (cantidad de filas, cantidad de columnas)
#tomar nota del total de registros que eso sirve para controlar el resultado de las tablas
dfa.shape

(8507472, 25)

In [ ]:
#para verificar el tamaño del listado de municipios con la agrupación en regiones
dfaux.shape

(135, 19)

In [ ]:
dfaux2.shape

(24, 16)

In [ ]:
#primera exploración de la base: lista de columnas
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8507472 entries, 0 to 8507471
Data columns (total 25 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   id_evento_caso                    int64  
 1   sexo                              object 
 2   edad                              float64
 3   edad_años_meses                   object 
 4   residencia_pais_nombre            object 
 5   residencia_provincia_nombre       object 
 6   residencia_departamento_nombre    object 
 7   carga_provincia_nombre            object 
 8   fecha_inicio_sintomas             object 
 9   fecha_apertura                    object 
 10  sepi_apertura                     int64  
 11  fecha_internacion                 object 
 12  cuidado_intensivo                 object 
 13  fecha_cui_intensivo               object 
 14  fallecido                         object 
 15  fecha_fallecimiento               object 
 16  asistencia_respiratoria_mecanica  ob

In [ ]:
dfaux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_Mun              135 non-null    int64 
 1   id_Prov_Mun         135 non-null    object
 2   Municipio_PBA       135 non-null    object
 3   Región              135 non-null    object
 4   AMBA                135 non-null    object
 5   Apertura_PBA_UNGS   135 non-null    object
 6   Apertura_PBA_UNDAV  135 non-null    object
 7   lat,long            135 non-null    object
 8   pob_mun_2015        135 non-null    int64 
 9   pob_mun_2016        135 non-null    int64 
 10  pob_mun_2017        135 non-null    int64 
 11  pob_mun_2018        135 non-null    int64 
 12  pob_mun_2019        135 non-null    int64 
 13  pob_mun_2020        135 non-null    int64 
 14  pob_mun_2021        135 non-null    int64 
 15  pob_mun_2022        135 non-null    int64 
 16  pob_mun_2023        135 no

In [ ]:
dfaux2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   cod_jur               24 non-null     int64 
 1   Jurisdiccion          24 non-null     object
 2   Cod ISO 3166 alpha 2  24 non-null     object
 3   Capital               23 non-null     object
 4   Gobernador            24 non-null     object
 5   pob_j_2015            24 non-null     int64 
 6   pob_j_2016            24 non-null     int64 
 7   pob_j_2017            24 non-null     int64 
 8   pob_j_2018            24 non-null     int64 
 9   pob_j_2019            24 non-null     int64 
 10  pob_j_2020            24 non-null     int64 
 11  pob_j_2021            24 non-null     int64 
 12  pob_j_2022            24 non-null     int64 
 13  pob_j_2023            24 non-null     int64 
 14  pob_j_2024            24 non-null     int64 
 15  pob_j_2025            24 non-null     int6

In [ ]:
#para ver el encabezado (y primeras 5 filas)
dfa.head(5)

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion
0,1000000,M,54.0,Años,Argentina,CABA,SIN ESPECIFICAR,Buenos Aires,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,6,Privado,Caso Descartado,Descartado,2,2020-06-09,0,2021-04-08
1,10000000,F,52.0,Años,Argentina,CABA,SIN ESPECIFICAR,CABA,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,2,Público,Caso Descartado,Descartado,2,2021-04-08,0,2021-04-08
2,10000001,F,22.0,Años,Argentina,Buenos Aires,Lomas de Zamora,Buenos Aires,2021-04-05,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso sospechoso - Activo Internado - Con muest...,Sospechoso,6,NaN,490,2021-04-08
3,10000002,F,43.0,Años,Argentina,Buenos Aires,La Matanza,CABA,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,2,Público,Caso Descartado,Descartado,6,2021-04-06,427,2021-04-08
4,10000003,F,33.0,Años,Argentina,CABA,COMUNA 01,CABA,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,2,Público,Caso Descartado,Descartado,2,2021-04-08,1,2021-04-08


In [ ]:
#tabla resumen que clasifica el total de casos en la base, a la fecha
dftotal=dfa.groupby('clasificacion_resumen')['id_evento_caso'].count()
#el siguiente total sirve para controlar que la función considere todos los registros
total_de_registros=dftotal.sum() 
print(dftotal)
print('Cuenta de registros=',total_de_registros)

clasificacion_resumen
Confirmado        2473750
Descartado        5319330
Sin Clasificar        175
Sospechoso         714217
Name: id_evento_caso, dtype: int64
Cuenta de registros= 8507472


In [ ]:
#entrando en el total de confirmados...
dfresumen1 = dfa.loc[dfa['clasificacion_resumen'].isin(['Confirmado'])].groupby('fallecido') ['id_evento_caso'].count()
Total1 = dfresumen1.sum()
print(dfresumen1)
print('Total',Total1)

fallecido
NO    2416628
SI      57122
Name: id_evento_caso, dtype: int64
Total 2473750


In [ ]:
#resto de tablas (auxiliares)

In [ ]:
dfaux.head(135)

,id_Mun,id_Prov_Mun,Municipio_PBA,Región,AMBA,Apertura_PBA_UNGS,Apertura_PBA_UNDAV,"lat,long",pob_mun_2015,pob_mun_2016,pob_mun_2017,pob_mun_2018,pob_mun_2019,pob_mun_2020,pob_mun_2021,pob_mun_2022,pob_mun_2023,pob_mun_2024,pob_mun_2025
0,7,6-7,Adolfo Alsina,Región I,Interior PBA,Interior PBA,Interior PBA,"-37.179483264541,-62.7599903900444",17356,17382,17407,17433,17458,17483,17507,17532,17556,17580,17603
1,14,6-14,Adolfo Gonzáles Chaves,Región I,Interior PBA,Interior PBA,Interior PBA,"-38.0323707296254,-60.0993919272956",12040,12016,11992,11968,11944,11922,11899,11877,11855,11832,11811
2,56,6-56,Bahía Blanca,Región I,Interior PBA,Interior PBA,Interior PBA,"-38.7227702774263,-62.2727419997435",307250,307836,308413,308982,309544,310095,310639,311175,311704,312224,312738
3,182,6-182,Coronel de Marina L. Rosales,Región I,Interior PBA,Interior PBA,Interior PBA,"-38.8781733536173,-62.0794804610544",63124,63203,63281,63358,63433,63508,63581,63654,63725,63795,63864
4,189,6-189,Coronel Dorrego,Región I,Interior PBA,Interior PBA,Interior PBA,"-38.7187998749142,-61.2883602471277",15589,15511,15435,15359,15285,15212,15141,15070,15000,14931,14864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,648,6-648,Presidente Perón,Región XI,AMBA,Tercer cordón,Tercer cordón,"-34.9179865320528,-58.3821570397156",93960,96420,98847,101239,103597,105918,108203,110457,112680,114868,117024
131,655,6-655,Punta Indio,Región XI,Interior PBA,Interior PBA,Interior PBA,"-35.3874096777144,-57.338171581753",10282,10343,10404,10463,10521,10579,10635,10692,10747,10802,10855
132,778,6-778,San Vicente,Región XI,AMBA,Tercer cordón,Tercer cordón,"-35.0259018141174,-58.4224036114375",68634,70388,72118,73825,75506,77161,78791,80398,81983,83544,85082
133,812,6-812,Tordillo,Región XI,Interior PBA,Interior PBA,Interior PBA,"-36.5208605076063,-57.3249387125515",1794,1796,1799,1801,1804,1807,1809,1812,1814,1816,1818


In [ ]:
dfaux2.head(5)

,cod_jur,Jurisdiccion,Cod ISO 3166 alpha 2,Capital,Gobernador,pob_j_2015,pob_j_2016,pob_j_2017,pob_j_2018,pob_j_2019,pob_j_2020,pob_j_2021,pob_j_2022,pob_j_2023,pob_j_2024,pob_j_2025
0,2,Ciudad Autónoma de Buenos Aires,AR-C,NaN,Horacio Rodríguez Larreta,3054267,3059122,3063728,3068043,3072029,3075646,3078836,3081550,3083770,3085483,3086680
1,6,Buenos Aires,AR-B,La Plata,Axel Kicillof,16659931,16841135,17020012,17196396,17370144,17541141,17709598,17875743,18039509,18200851,18359753
2,10,Catamarca,AR-K,San Fernando del Valle de Catamarca,Raúl Jalil,396895,400678,404433,408152,411824,415438,418991,422476,425885,429208,432438
3,14,Córdoba,AR-X,Córdoba,Juan Schiaretti,3567654,3606540,3645321,3683937,3722332,3760450,3798261,3835738,3872830,3909489,3945677
4,18,Corrientes,AR-W,Corrientes,Gustavo Valdés,1070283,1080655,1090938,1101084,1111052,1120801,1130320,1139604,1148631,1157386,1165860


In [ ]:
dfvacc1.head(5)

,jurisdiccion_codigo_indec,jurisdiccion_nombre,vacuna_nombre,primera_dosis_cantidad,segunda_dosis_cantidad
0,6,Buenos Aires,AstraZeneca ChAdOx1 S recombinante,13016,0
1,6,Buenos Aires,COVISHIELD ChAdOx1nCoV COVID 19,218925,0
2,6,Buenos Aires,Sinopharm Vacuna SARSCOV 2 inactivada,360990,0
3,6,Buenos Aires,Sputnik V COVID19 Instituto Gamaleya,932255,267179
4,2,CABA,AstraZeneca ChAdOx1 S recombinante,17368,1


## **Elaboración de tablas**

###Eliminación de variables que no se usarán

In [ ]:
#soltar variables que no se usarán de la base con información de los municipios
dfaux.drop(columns=['pob_mun_2015','pob_mun_2016','pob_mun_2017','pob_mun_2018','pob_mun_2019','pob_mun_2022','pob_mun_2023','pob_mun_2024','pob_mun_2025'],axis=1,inplace=True)

In [ ]:
#soltar variables que no se usarán de la base con información de las provincias
dfaux2.drop(columns=['Gobernador','pob_j_2015','pob_j_2016','pob_j_2017','pob_j_2018','pob_j_2019','pob_j_2022','pob_j_2023','pob_j_2024','pob_j_2025'],axis=1,inplace=True)

In [ ]:
#para trabajar solamente con la base con los confirmados
df = dfa.loc[dfa['clasificacion_resumen'].isin(['Confirmado'])]

In [ ]:
df.shape

(2473750, 25)

### Agrupamiento y agregado de categorías en municipios

In [ ]:
#para homogeneizar el id de cada jurisdicción y poder vincular las tablas
dfvacc1.rename({'jurisdiccion_codigo_indec':'cod_jur'},axis=1,inplace=True)
#para convertir la variable en una de texto. 
dfvacc1['cod_jur']=dfvacc1['cod_jur'].astype(str) 
dfvacc=dfvacc1.groupby(['cod_jur']).agg(
    primera_dosis_cantidad=('primera_dosis_cantidad','sum'),
    segunda_dosis_cantidad=('segunda_dosis_cantidad','sum')
)

In [ ]:
#esto es para convertir la variable en una de texto (se unirán tablas por variables que tienen que ser compatibles)
df['residencia_provincia_id']=df['residencia_provincia_id'].astype(str) 
df['carga_provincia_id']=df['carga_provincia_id'].astype(str) 
df['residencia_departamento_id']=df['residencia_departamento_id'].astype(str)
dfaux['id_Prov_Mun']=dfaux['id_Prov_Mun'].astype(str) 
dfaux2['cod_jur']=dfaux2['cod_jur'].astype(str) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
#para obtener una nueva columna donde se indique la provincia de residencia o la carga en el caso de que no esté especificada.
df['cod_jur'] = np.where(df['residencia_provincia_id']=='99',df['carga_provincia_id'],df['residencia_provincia_id'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df.head(50)

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion,cod_jur
27,10000026,F,23.0,Años,Argentina,Córdoba,Capital,Córdoba,2021-04-03,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,14,Público,Caso confirmado por laboratorio - Activo,Confirmado,14,2021-04-05,14,2021-04-08,14
60,10000057,M,35.0,Años,Argentina,Buenos Aires,Mercedes,Buenos Aires,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-06,532,2021-04-08,6
69,10000066,F,43.0,Años,Argentina,Buenos Aires,Berazategui,Buenos Aires,2021-04-02,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por criterio clinico-epidemiol...,Confirmado,6,NaN,91,2021-04-08,6
73,1000007,M,26.0,Años,Argentina,Formosa,Patiño,Formosa,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,34,Público,Caso confirmado por laboratorio - No Activo po...,Confirmado,34,2020-06-25,35,2021-04-08,34
76,10000072,F,37.0,Años,Argentina,Santa Fe,San Lorenzo,Santa Fe,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,82,Público,Caso confirmado por laboratorio - Activo,Confirmado,82,2021-04-07,119,2021-04-08,82
105,10000099,F,53.0,Años,Argentina,Buenos Aires,Pilar,Buenos Aires,2021-04-03,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-06,638,2021-04-08,6
106,1000010,M,8.0,Años,Argentina,CABA,COMUNA 07,CABA,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,2,Público,Caso confirmado por laboratorio - No activo (p...,Confirmado,2,2020-06-01,7,2021-04-08,2
116,10000109,F,26.0,Años,Argentina,San Luis,La Capital,San Luis,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,74,Público,Caso confirmado por laboratorio - Activo,Confirmado,74,2021-04-08,56,2021-04-08,74
128,1000012,M,46.0,Años,Argentina,CABA,SIN ESPECIFICAR,CABA,2020-05-26,2020-06-01,23,2020-05-31,NO,NaN,NO,NaN,NO,2,Privado,Caso confirmado por laboratorio - No Activo po...,Confirmado,2,2020-05-31,0,2021-04-08,2
135,10000127,F,20.0,Años,Argentina,Buenos Aires,Quilmes,CABA,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,2,Público,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-07,658,2021-04-08,6


In [ ]:
df['cod_jur']=df['cod_jur'].astype(str) #para convertir la nueva variable en texto.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#genero un cod(concatenado PBA y Mun)= cod_provincia + cod_municipio
#esto es para convertir la variable en una de texto
df['id_Prov_Mun']=df['cod_jur']+'-'+df['residencia_departamento_id']
df['id_Prov_Mun']=df['id_Prov_Mun'].astype(str) #esto es para convertir la variable en una de texto

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df.head(5)

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion,cod_jur,id_Prov_Mun
27,10000026,F,23.0,Años,Argentina,Córdoba,Capital,Córdoba,2021-04-03,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,14,Público,Caso confirmado por laboratorio - Activo,Confirmado,14,2021-04-05,14,2021-04-08,14,14-14
60,10000057,M,35.0,Años,Argentina,Buenos Aires,Mercedes,Buenos Aires,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-06,532,2021-04-08,6,6-532
69,10000066,F,43.0,Años,Argentina,Buenos Aires,Berazategui,Buenos Aires,2021-04-02,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por criterio clinico-epidemiol...,Confirmado,6,NaN,91,2021-04-08,6,6-91
73,1000007,M,26.0,Años,Argentina,Formosa,Patiño,Formosa,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,34,Público,Caso confirmado por laboratorio - No Activo po...,Confirmado,34,2020-06-25,35,2021-04-08,34,34-35
76,10000072,F,37.0,Años,Argentina,Santa Fe,San Lorenzo,Santa Fe,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,82,Público,Caso confirmado por laboratorio - Activo,Confirmado,82,2021-04-07,119,2021-04-08,82,82-119


In [ ]:
#para obtener una nueva base mixta=que ahora tiene las regiones (y otras descripciones de municipios, para control)
#la base original se expande con las descripciones adicionales de los municipios de BA y la región sanitaria
df1p = pd.merge(df, 
               dfaux, 
               on ='id_Prov_Mun', 
               how ='left')

In [ ]:
#para obtener una nueva base mixta=que ahora tiene las provincias (contiene los no especidificados con carga en esa jurisdicción)
dfaux22 = pd.merge(dfaux2, 
               dfvacc, 
               on ='cod_jur', 
               how ='left')
dfaux22

,cod_jur,Jurisdiccion,Cod ISO 3166 alpha 2,Capital,pob_j_2020,pob_j_2021,primera_dosis_cantidad,segunda_dosis_cantidad
0,2,Ciudad Autónoma de Buenos Aires,AR-C,NaN,3075646,3078836,399006,84396
1,6,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179
2,10,Catamarca,AR-K,San Fernando del Valle de Catamarca,415438,418991,40516,6291
3,14,Córdoba,AR-X,Córdoba,3760450,3798261,379636,50255
4,18,Corrientes,AR-W,Corrientes,1120801,1130320,85294,15023
5,22,Chaco,AR-H,Resistencia,1204541,1216247,106050,16424
6,26,Chubut,AR-U,Rawson,618994,629181,51822,8145
7,30,Entre Ríos,AR-E,Paraná,1385961,1398510,104986,19496
8,34,Formosa,AR-P,Formosa,605193,610019,58619,7974
9,38,Jujuy,AR-Y,San Salvador de Jujuy,770881,779212,63856,13830


In [ ]:
#para obtener una nueva base mixta=que ahora tiene las provincias (contiene los no especidificados con carga en esa jurisdicción)
df1 = pd.merge(df1p, 
               dfaux22, 
               on ='cod_jur', 
               how ='left')
df1

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion,cod_jur,id_Prov_Mun,id_Mun,Municipio_PBA,Región,AMBA,Apertura_PBA_UNGS,Apertura_PBA_UNDAV,"lat,long",pob_mun_2020,pob_mun_2021,Jurisdiccion,Cod ISO 3166 alpha 2,Capital,pob_j_2020,pob_j_2021,primera_dosis_cantidad,segunda_dosis_cantidad
0,10000026,F,23.0,Años,Argentina,Córdoba,Capital,Córdoba,2021-04-03,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,14,Público,Caso confirmado por laboratorio - Activo,Confirmado,14,2021-04-05,14,2021-04-08,14,14-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Córdoba,AR-X,Córdoba,3760450,3798261,379636,50255
1,10000057,M,35.0,Años,Argentina,Buenos Aires,Mercedes,Buenos Aires,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-06,532,2021-04-08,6,6-532,532.0,Mercedes,Región X,Interior PBA,Interior PBA,Interior PBA,"-34.6507332135669,-59.4313874409776",67793.0,68165.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179
2,10000066,F,43.0,Años,Argentina,Buenos Aires,Berazategui,Buenos Aires,2021-04-02,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por criterio clinico-epidemiol...,Confirmado,6,NaN,91,2021-04-08,6,6-91,91.0,Berazategui,Región VI,AMBA,Segundo cordón,Segundo cordón,"-34.7630163220456,-58.2121691470884",365771.0,369294.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179
3,1000007,M,26.0,Años,Argentina,Formosa,Patiño,Formosa,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,34,Público,Caso confirmado por laboratorio - No Activo po...,Confirmado,34,2020-06-25,35,2021-04-08,34,34-35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Formosa,AR-P,Formosa,605193,610019,58619,7974
4,10000072,F,37.0,Años,Argentina,Santa Fe,San Lorenzo,Santa Fe,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,82,Público,Caso confirmado por laboratorio - Activo,Confirmado,82,2021-04-07,119,2021-04-08,82,82-119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Santa Fe,AR-S,Santa Fe,3536418,3563390,300481,55845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473745,999995,M,34.0,Años,Argentina,Formosa,Pilcomayo,Formosa,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,34,Público,Caso confirmado por laboratorio - No Activo po...,Confirmado,34,2020-06-12,49,2021-04-08,34,34-49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Formosa,AR-P,Formosa,605193,610019,58619,7974
2473746,9999957,F,23.0,Años,Argentina,Buenos Aires,General Rodríguez,Buenos Aires,2021-04-02,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por criterio clinico-epidemiol...,Confirmado,6,NaN,364,2021-04-08,6,6-364,364.0,General Rodríguez,Región VII,AMBA,Tercer cordón,Tercer cordón,"-34.6064310690385,-58.9521606355049",109695.0,111759.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179
2473747,9999959,M,52.0,Años,Argentina,Buenos Aires,Moreno,Buenos Aires,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Privado,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-07,560,2021-04-08,6,6-560,560.0,Moreno,Región VII,AMBA,Segundo cordón,Segundo cordón,"-34.6512923274789,-58.7900019150563",541691.0,549930.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179
2473748,9999966,M,64.0,Años,Argentina,Buenos Aires,Merlo,Buenos Aires,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Privado,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-06,539,2021-04-08,6,6-539,539.0,Merlo,Región VII,AMBA,Segundo cordón,Segundo cordón,"-34.6649832276938,-58.7279481147137",606413.0,613509.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,

In [ ]:
df1.head(5)

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion,cod_jur,id_Prov_Mun,id_Mun,Municipio_PBA,Región,AMBA,Apertura_PBA_UNGS,Apertura_PBA_UNDAV,"lat,long",pob_mun_2020,pob_mun_2021,Jurisdiccion,Cod ISO 3166 alpha 2,Capital,pob_j_2020,pob_j_2021,primera_dosis_cantidad,segunda_dosis_cantidad
0,10000026,F,23.0,Años,Argentina,Córdoba,Capital,Córdoba,2021-04-03,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,14,Público,Caso confirmado por laboratorio - Activo,Confirmado,14,2021-04-05,14,2021-04-08,14,14-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Córdoba,AR-X,Córdoba,3760450,3798261,379636,50255
1,10000057,M,35.0,Años,Argentina,Buenos Aires,Mercedes,Buenos Aires,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-06,532,2021-04-08,6,6-532,532.0,Mercedes,Región X,Interior PBA,Interior PBA,Interior PBA,"-34.6507332135669,-59.4313874409776",67793.0,68165.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179
2,10000066,F,43.0,Años,Argentina,Buenos Aires,Berazategui,Buenos Aires,2021-04-02,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por criterio clinico-epidemiol...,Confirmado,6,NaN,91,2021-04-08,6,6-91,91.0,Berazategui,Región VI,AMBA,Segundo cordón,Segundo cordón,"-34.7630163220456,-58.2121691470884",365771.0,369294.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179
3,1000007,M,26.0,Años,Argentina,Formosa,Patiño,Formosa,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,34,Público,Caso confirmado por laboratorio - No Activo po...,Confirmado,34,2020-06-25,35,2021-04-08,34,34-35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Formosa,AR-P,Formosa,605193,610019,58619,7974
4,10000072,F,37.0,Años,Argentina,Santa Fe,San Lorenzo,Santa Fe,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,82,Público,Caso confirmado por laboratorio - Activo,Confirmado,82,2021-04-07,119,2021-04-08,82,82-119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Santa Fe,AR-S,Santa Fe,3536418,3563390,300481,55845


In [ ]:
df1.shape

(2473750, 43)

In [ ]:
#creo una columna que me traiga el nombre de distrito (no sólo los de PBA): es igual a la columna 'Municipio_PBA' -que trae sólo los municipios de PBA-, en caso de que se trate de otra provincia me indica la columna 'residencia_departamento_nombre'.
df1['Distrito'] = df1['Municipio_PBA'].fillna(df1['residencia_departamento_nombre']) 

In [ ]:
df1.head(5)

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion,cod_jur,id_Prov_Mun,id_Mun,Municipio_PBA,Región,AMBA,Apertura_PBA_UNGS,Apertura_PBA_UNDAV,"lat,long",pob_mun_2020,pob_mun_2021,Jurisdiccion,Cod ISO 3166 alpha 2,Capital,pob_j_2020,pob_j_2021,primera_dosis_cantidad,segunda_dosis_cantidad,Distrito
0,10000026,F,23.0,Años,Argentina,Córdoba,Capital,Córdoba,2021-04-03,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,14,Público,Caso confirmado por laboratorio - Activo,Confirmado,14,2021-04-05,14,2021-04-08,14,14-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Córdoba,AR-X,Córdoba,3760450,3798261,379636,50255,Capital
1,10000057,M,35.0,Años,Argentina,Buenos Aires,Mercedes,Buenos Aires,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-06,532,2021-04-08,6,6-532,532.0,Mercedes,Región X,Interior PBA,Interior PBA,Interior PBA,"-34.6507332135669,-59.4313874409776",67793.0,68165.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179,Mercedes
2,10000066,F,43.0,Años,Argentina,Buenos Aires,Berazategui,Buenos Aires,2021-04-02,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por criterio clinico-epidemiol...,Confirmado,6,NaN,91,2021-04-08,6,6-91,91.0,Berazategui,Región VI,AMBA,Segundo cordón,Segundo cordón,"-34.7630163220456,-58.2121691470884",365771.0,369294.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179,Berazategui
3,1000007,M,26.0,Años,Argentina,Formosa,Patiño,Formosa,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,34,Público,Caso confirmado por laboratorio - No Activo po...,Confirmado,34,2020-06-25,35,2021-04-08,34,34-35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Formosa,AR-P,Formosa,605193,610019,58619,7974,Patiño
4,10000072,F,37.0,Años,Argentina,Santa Fe,San Lorenzo,Santa Fe,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,82,Público,Caso confirmado por laboratorio - Activo,Confirmado,82,2021-04-07,119,2021-04-08,82,82-119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Santa Fe,AR-S,Santa Fe,3536418,3563390,300481,55845,San Lorenzo


In [ ]:
df1.shape

(2473750, 44)

In [ ]:
#creo una columna con la fecha como en el pivot (fecha diagnóstico, y en caso de vacío: fecha de apertura)
df1['fecha_calculada'] = df1['fecha_diagnostico'].fillna(df1['fecha_apertura']) 

In [ ]:
df1.head(5)

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion,cod_jur,id_Prov_Mun,id_Mun,Municipio_PBA,Región,AMBA,Apertura_PBA_UNGS,Apertura_PBA_UNDAV,"lat,long",pob_mun_2020,pob_mun_2021,Jurisdiccion,Cod ISO 3166 alpha 2,Capital,pob_j_2020,pob_j_2021,primera_dosis_cantidad,segunda_dosis_cantidad,Distrito,fecha_calculada
0,10000026,F,23.0,Años,Argentina,Córdoba,Capital,Córdoba,2021-04-03,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,14,Público,Caso confirmado por laboratorio - Activo,Confirmado,14,2021-04-05,14,2021-04-08,14,14-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Córdoba,AR-X,Córdoba,3760450,3798261,379636,50255,Capital,2021-04-05
1,10000057,M,35.0,Años,Argentina,Buenos Aires,Mercedes,Buenos Aires,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-06,532,2021-04-08,6,6-532,532.0,Mercedes,Región X,Interior PBA,Interior PBA,Interior PBA,"-34.6507332135669,-59.4313874409776",67793.0,68165.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179,Mercedes,2021-04-06
2,10000066,F,43.0,Años,Argentina,Buenos Aires,Berazategui,Buenos Aires,2021-04-02,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por criterio clinico-epidemiol...,Confirmado,6,NaN,91,2021-04-08,6,6-91,91.0,Berazategui,Región VI,AMBA,Segundo cordón,Segundo cordón,"-34.7630163220456,-58.2121691470884",365771.0,369294.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179,Berazategui,2021-04-08
3,1000007,M,26.0,Años,Argentina,Formosa,Patiño,Formosa,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,34,Público,Caso confirmado por laboratorio - No Activo po...,Confirmado,34,2020-06-25,35,2021-04-08,34,34-35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Formosa,AR-P,Formosa,605193,610019,58619,7974,Patiño,2020-06-25
4,10000072,F,37.0,Años,Argentina,Santa Fe,San Lorenzo,Santa Fe,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,82,Público,Caso confirmado por laboratorio - Activo,Confirmado,82,2021-04-07,119,2021-04-08,82,82-119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Santa Fe,AR-S,Santa Fe,3536418,3563390,300481,55845,San Lorenzo,2021-04-07


In [ ]:
df1.shape

(2473750, 45)

In [ ]:
df1.head(5)

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion,cod_jur,id_Prov_Mun,id_Mun,Municipio_PBA,Región,AMBA,Apertura_PBA_UNGS,Apertura_PBA_UNDAV,"lat,long",pob_mun_2020,pob_mun_2021,Jurisdiccion,Cod ISO 3166 alpha 2,Capital,pob_j_2020,pob_j_2021,primera_dosis_cantidad,segunda_dosis_cantidad,Distrito,fecha_calculada
0,10000026,F,23.0,Años,Argentina,Córdoba,Capital,Córdoba,2021-04-03,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,14,Público,Caso confirmado por laboratorio - Activo,Confirmado,14,2021-04-05,14,2021-04-08,14,14-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Córdoba,AR-X,Córdoba,3760450,3798261,379636,50255,Capital,2021-04-05
1,10000057,M,35.0,Años,Argentina,Buenos Aires,Mercedes,Buenos Aires,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por laboratorio - Activo,Confirmado,6,2021-04-06,532,2021-04-08,6,6-532,532.0,Mercedes,Región X,Interior PBA,Interior PBA,Interior PBA,"-34.6507332135669,-59.4313874409776",67793.0,68165.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179,Mercedes,2021-04-06
2,10000066,F,43.0,Años,Argentina,Buenos Aires,Berazategui,Buenos Aires,2021-04-02,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,6,Público,Caso confirmado por criterio clinico-epidemiol...,Confirmado,6,NaN,91,2021-04-08,6,6-91,91.0,Berazategui,Región VI,AMBA,Segundo cordón,Segundo cordón,"-34.7630163220456,-58.2121691470884",365771.0,369294.0,Buenos Aires,AR-B,La Plata,17541141,17709598,1525186,267179,Berazategui,2021-04-08
3,1000007,M,26.0,Años,Argentina,Formosa,Patiño,Formosa,NaN,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,34,Público,Caso confirmado por laboratorio - No Activo po...,Confirmado,34,2020-06-25,35,2021-04-08,34,34-35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Formosa,AR-P,Formosa,605193,610019,58619,7974,Patiño,2020-06-25
4,10000072,F,37.0,Años,Argentina,Santa Fe,San Lorenzo,Santa Fe,NaN,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,82,Público,Caso confirmado por laboratorio - Activo,Confirmado,82,2021-04-07,119,2021-04-08,82,82-119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Santa Fe,AR-S,Santa Fe,3536418,3563390,300481,55845,San Lorenzo,2021-04-07


### Nuevas variables

In [ ]:
#para obtener una nueva variable que indica si el caso trata de alguien mayor a 60 años
df1['+60']=np.where(df1['edad']>=60,'60 años o +',
                    np.where(df1['edad']<18,'Menores de 18','18 a 59 años'))           

In [ ]:
#para agrupar los casos confirmados, creo una nueva variable resumen a partir de 'clasificacion'
df1['agrupamiento_confirmados']=df1['clasificacion'].replace({'Caso confirmado por criterio clinico-epidemiológico - Activo':'Activo', 
                    'Caso confirmado por criterio clínico-epidemiológico  - No activo (por tiempo de evolución)':'Recuperado', 
                    'Caso confirmado por criterio clínico - epidemiológico -  Activo internado':'Activo - Internado',
                    'Caso confirmado por criterio clínico-epidemiologico - Fallecido':'Fallecido', 
                    'Caso confirmado por laboratorio - Activo':'Activo', 
                    'Caso confirmado por laboratorio - Activo Internado':'Activo - Internado', 
                    'Caso confirmado por laboratorio - Fallecido':'Fallecido',
                    'Caso confirmado por laboratorio - No Activo por criterio de laboratorio':'Recuperado',
                    'Caso confirmado por laboratorio - No activo (por tiempo de evolución)':'Recuperado'}, inplace=False)

In [ ]:
#para obtener un recuento de los fallecidos
df1['cuenta_fallecidos']=np.where(df1['fallecido']=='SI',1,0)

### Primeras tablas / tablas de control

In [ ]:
#tabla resumen con confirmados por provincia
dfresumen2_2 = df1.loc[df1['clasificacion_resumen'].isin(['Confirmado'])].groupby('Jurisdiccion') ['id_evento_caso'].count()
Totalprueba2_2 = dfresumen2_2.sum()
print(dfresumen2_2)
print('Total',Totalprueba2_2)

Jurisdiccion
Buenos Aires                                             1049980
Catamarca                                                  11287
Chaco                                                      39784
Chubut                                                     51013
Ciudad Autónoma de Buenos Aires                           277993
Corrientes                                                 32341
Córdoba                                                   191053
Entre Ríos                                                 51969
Formosa                                                     2541
Jujuy                                                      22890
La Pampa                                                   22256
La Rioja                                                   11714
Mendoza                                                    77790
Misiones                                                   12868
Neuquén                                                    66173
Río Negro   

In [ ]:
#tabla resumen con fallecidos por provincia
dfresumen2_3 = df1.loc[df1['clasificacion_resumen'].isin(['Confirmado'])].loc[df1['fallecido'].isin(['SI'])].groupby(['Jurisdiccion']) ['id_evento_caso'].count()
Totalprueba2_3 = dfresumen2_3.sum()
print(dfresumen2_3)
print('Total',Totalprueba2_3)

Jurisdiccion
Buenos Aires                                             28960
Catamarca                                                   18
Chaco                                                      957
Chubut                                                     868
Ciudad Autónoma de Buenos Aires                           7034
Corrientes                                                 334
Córdoba                                                   3095
Entre Ríos                                                 959
Formosa                                                     46
Jujuy                                                      978
La Pampa                                                   328
La Rioja                                                   450
Mendoza                                                   1528
Misiones                                                   216
Neuquén                                                   1011
Río Negro                                 

In [ ]:
#tabla confirmados PBA + fallecidos PBA
dfresumen2_4 = df1.loc[df1['cod_jur'].isin(['6'])].loc[df1['clasificacion_resumen'].isin(['Confirmado'])].groupby(['AMBA'],dropna=False) ['id_evento_caso'].count()
dfresumen2_5 = df1.loc[df1['cod_jur'].isin(['6'])].loc[df1['clasificacion_resumen'].isin(['Confirmado'])].loc[df1['fallecido'].isin(['SI'])].groupby(['AMBA'],dropna=False) ['id_evento_caso'].count()
Totalprueba2_4 = dfresumen2_4.sum()
Totalprueba2_5 = dfresumen2_5.sum()
print(dfresumen2_4)
print('Total',Totalprueba2_4)
print(dfresumen2_5)
print('Total',Totalprueba2_5)

AMBA
AMBA            817124
Interior PBA    229160
NaN               3696
Name: id_evento_caso, dtype: int64
Total 1049980
AMBA
AMBA            22995
Interior PBA     5857
NaN               108
Name: id_evento_caso, dtype: int64
Total 28960


In [ ]:
#tabla resumen con encabezados de provincia, clasificación de caso, detalle de caso (para ver los activos), fallecido, regiones y recuento de casos
dfresumen4=df1.groupby(['Jurisdiccion','clasificacion_resumen','clasificacion','fallecido','Región','AMBA'],dropna=False)['id_evento_caso'].count()
Total_de_registros4=dfresumen4.sum()
print(dfresumen4)
print('Total de control',Total_de_registros4)

Jurisdiccion  clasificacion_resumen  clasificacion                                                            fallecido  Región      AMBA        
Buenos Aires  Confirmado             Caso confirmado por criterio clinico-epidemiol¿gico - Activo             NO         Región I    Interior PBA      258
                                                                                                                         Región II   Interior PBA      179
                                                                                                                         Región III  Interior PBA      101
                                                                                                                         Región IV   Interior PBA       62
                                                                                                                         Región IX   Interior PBA       21
                                                                               

### Elaboración de tablas resumen

In [ ]:
#tabla resumen final ordenada por fecha y encabezados de provincia, municipio, clasificación de caso completa, regiones y recuento de casos confirmados y fallecidos
df1['fecha_calculada']=df1['fecha_calculada'].astype(str) #esto es para convertir la variable de fecha a una variable de texto
dfresumenfinalea=df1.loc[df1['clasificacion_resumen'].isin(['Confirmado'])] #esto es para seleccionar únicamente los casos confirmados
dfresumenfinal=dfresumenfinalea.groupby(['fecha_calculada','Jurisdiccion','Distrito','+60','agrupamiento_confirmados','Región','AMBA','Apertura_PBA_UNGS'],dropna=False).agg(
    casos_confirmados=('id_evento_caso','count'),
    fallecidos=('cuenta_fallecidos','sum')
)

Total_registros_control=dfresumenfinal.sum()
print(dfresumenfinal)
print('Chequeo final',Total_registros_control)

                                                                                                               casos_confirmados  fallecidos
fecha_calculada Jurisdiccion Distrito    +60           agrupamiento_confirmados Región AMBA Apertura_PBA_UNGS                               
2002-10-09      Neuquén      Confluencia 18 a 59 años  Recuperado               NaN    NaN  NaN                                1           0
2002-10-11      Neuquén      Confluencia 18 a 59 años  Recuperado               NaN    NaN  NaN                                1           0
2002-10-21      Neuquén      Confluencia 60 años o +   Recuperado               NaN    NaN  NaN                                1           0
2002-11-09      San Juan     Chimbas     18 a 59 años  Recuperado               NaN    NaN  NaN                                1           0
2012-09-21      Córdoba      Capital     Menores de 18 Recuperado               NaN    NaN  NaN                                1           0
...          

In [ ]:
#tabla resumen final por provincia, población y recuento de casos confirmados y fallecidos
dfresumen7=dfresumenfinalea.groupby(['Jurisdiccion','Cod ISO 3166 alpha 2','pob_j_2021','primera_dosis_cantidad','segunda_dosis_cantidad'],dropna=False).agg(
    casos_confirmados=('id_evento_caso','count'),
    fallecidos=('cuenta_fallecidos','sum')
)

Total_registros_control2=dfresumen7.sum()
print(dfresumen7)
print('Chequeo final',Total_registros_control2)

                                                                                                                                  casos_confirmados  fallecidos
Jurisdiccion                                       Cod ISO 3166 alpha 2 pob_j_2021 primera_dosis_cantidad segunda_dosis_cantidad                               
Buenos Aires                                       AR-B                 17709598   1525186                267179                            1049980       28960
Catamarca                                          AR-K                 418991     40516                  6291                                11287          18
Chaco                                              AR-H                 1216247    106050                 16424                               39784         957
Chubut                                             AR-U                 629181     51822                  8145                                51013         868
Ciudad Autónoma de Buenos Aires         

In [ ]:
#tabla resumen final por municipio de la PBA, población y recuento de casos confirmados y fallecidos
dfresumenfinaleb=dfresumenfinalea.loc[dfresumenfinalea['Jurisdiccion'].isin(['Buenos Aires'])] #esto es para seleccionar únicamente los casos de PBA
dfresumen8=dfresumenfinaleb.groupby(['Distrito','Región','AMBA','Apertura_PBA_UNGS','lat,long','pob_mun_2021'],dropna=False).agg(
    casos_confirmados=('id_evento_caso','count'),
    fallecidos=('cuenta_fallecidos','sum')
)

#Total_registros_control3=dfresumen8.sum()
#print(dfresumen8)
#print('Chequeo final',Total_registros_control3)

## **Guardado / descarga**

In [ ]:
#si se guarda en ubicación en drive como csv ('montar' e indicar ubicación en propio drive)
#esta es el resumen ordenado por fecha (verificar**)
dfresumenfinal.to_csv('/content/drive/MyDrive/Proyectos/COVID/tabladiaria.csv',header = True,index=True,sep=",")
#tablas resumen
dfresumen7.to_csv('/content/drive/MyDrive/Proyectos/COVID/Resumen_ARG.csv',header = True,index=True,sep=",")
dfresumen8.to_csv('/content/drive/MyDrive/Proyectos/COVID/Resumen_PBA.csv',header = True,index=True,sep=",")

## **Pasos que faltan:**
  Confirmar subtotales por provincia y municipio (OK en duplicado Power BI y en PIVOT, restan controles cruzados...). Para los casos no especificados se considera el distrito de carga.

  Constatar la fecha que se utiliza en pivot (OK con PIVOT, restan controles cruzados)



## **Pruebas/controles**

In [ ]:
#dfprueba.shape

In [ ]:
#dfprueba11=df1.groupby(['residencia_provincia_id','residencia_provincia_nombre'])['id_evento_caso'].count()
#totaldfprueba11=dfprueba11.sum()
#print(dfprueba11)
#print('Total',totaldfprueba11)

In [ ]:
#dfprueba2 = df1.loc[df1['residencia_provincia_id'].isin(['6'])].groupby(['residencia_departamento_id','residencia_departamento_nombre']) ['id_evento_caso'].count()
#Totalprueba2 = dfprueba2.sum()
#print(dfprueba2)
#print('Total',Totalprueba2)

In [ ]:
#dfprueba2_2 = df1.loc[df1['residencia_provincia_id'].isin(['6'])].groupby(['residencia_departamento_id','residencia_departamento_nombre','Municipio']) ['id_evento_caso'].count()
#Totalprueba2_2 = dfprueba2.sum()
#print(dfprueba2_2)
#print('Total',Totalprueba2_2)

In [ ]:
#Diferencia=Totalprueba2-Totalprueba2_2
#print(Diferencia)

In [ ]:
#dfprueba2=df1.groupby(['residencia_provincia_id','residencia_departamento_nombre','residencia_departamento_id'],dropna=False)['id_evento_caso'].count()
#dfprueba22=dfprueba2.['residencia_provincia_id'].isin(['6'])
#totaldfprueba2=dfprueba2.sum()
#dfprueba22=dfprueba2['residencia_provincia_id'].isin(['6'])
#print(totaldfprueba2)
#dfprueba2.head(200)

In [ ]:
#dfprueba2=df1['residencia_provincia_id'].isin(['6'])
#dfprueba3=df1.groupby(['Municipio','residencia_departamento_nombre'],dropna=False)['id_evento_caso'].count()
#totaldfprueba3=dfprueba3.sum()
#print(dfprueba3)

In [ ]:
#dfprueba2.head(400)

In [ ]:
#dfprueba=pd.pivot_table(df1,index=['residencia_provincia_id','residencia_departamento_id','id_Prov_Mun','residencia_provincia_nombre','Municipio','residencia_departamento_nombre','Región'],values='id_evento_caso',aggfunc='count')
#Totalprueba1 = dfprueba.sum()
#print(dfprueba)
#print('Total',Totalprueba1)

## **Líneas sin uso**

In [ ]:
#tabla resumen con total de casos agrupada
#dfresumenfinal=dfresumenfinalea.groupby(['fecha_calculada','Jurisdiccion','Distrito','sexo','+60','clasificacion','agrupamiento_confirmados','fallecido','Región','AMBA','Apertura_PBA_UNGS','Apertura_PBA_UNDAV','lat,long','Cod ISO 3166 alpha 2'],dropna=False)['id_evento_caso'].count()

In [ ]:
#gráfico1 preliminar
#dftotal.plot(kind='pie', legend=False)

In [ ]:
#gráfico 1 preliminar
#dfresumen1.plot(kind='pie', legend=False)

In [ ]:
#tabla resumen con encabezados de provincia, clasificación de caso, detalle de caso (para ver los activos), fallecido, región y recuento de casos (se filtran filas con N/A)
#dfresumenx=pd.pivot_table(df1,index=['residencia_provincia_nombre','clasificacion_resumen','clasificacion','fallecido','Región'],values='id_evento_caso',aggfunc='count')

In [ ]:
#resultado equivalente pero sin filtrar los casos confirmados:
#dfresumen2=pd.pivot_table(df1,index=['clasificacion_resumen','fallecido'],values='id_evento_caso',aggfunc='count')
#Total_de_control=dfresumen2.sum()
#print(dfresumen2)
#print('Total de control',Total_de_control)

In [ ]:
#tabla resumen con encabezados de provincia, clasificación de caso y recuento de casos
#dfresumen3_2=pd.pivot_table(df1,index=['residencia_provincia_nombre','clasificacion_resumen'],values='id_evento_caso',aggfunc='count')
#Total_de_registros2=dfresumen3_2.sum() 
#print(dfresumen3_2)
#print('Total de control',Total_de_registros2)

In [ ]:
#tabla resumen con encabezados de provincia, clasificación de caso, fallecido y recuento de casos
#dfresumen3=pd.pivot_table(df,index=['residencia_provincia_nombre','clasificacion_resumen','fallecido'],values='id_evento_caso',aggfunc='count')
#Total_de_registros2=dfresumen3.sum() 
#print(dfresumen3)
#print('Total de control',Total_de_registros2)

In [ ]:
#dfresumen2_2.plot(kind='pie', legend=False)

In [ ]:
#df1['Distrito']= np.where(df1['Municipio_PBA']==np.nan,df1['residencia_departamento_nombre'],df1['Municipio_PBA'])

In [ ]:
#df1['fecha_calculada']= np.where(df1['fecha_diagnostico']==np.nan,df1['fecha_apertura'],df1['fecha_diagnostico'])

In [ ]:
#tabla resumen con encabezados por jurisdicción/departamento, clasificación de caso, detalle de caso (para ver los activos),por fecha y recuento de casos
#dfresumen6ea=df1.loc[df1['clasificacion_resumen'].isin(['Confirmado'])] #esto es para seleccionar únicamente los casos confirmados
#dfresumen6=dfresumen6ea.groupby(['Jurisdiccion','Distrito','sexo','+60','clasificacion','agrupamiento_confirmados','Región','AMBA','Apertura_PBA_UNGS','Apertura_PBA_UNDAV','lat,long','pob_j_2021','pob_mun_2021'],dropna=False).agg(
#    casos_confirmados=('id_evento_caso','count'),
#    fallecidos=('cuenta_fallecidos','sum')
#)

#Total_de_registros6=dfresumen6.sum()
#print(dfresumen6)
#print('Total de control',Total_de_registros5)

In [ ]:
#tabla resumen con encabezados de provincia, clasificación de caso, detalle de caso (para ver los activos),por fecha y recuento de casos
#dfresumen5ea=df1.loc[df1['clasificacion_resumen'].isin(['Confirmado'])] #esto es para seleccionar únicamente los casos confirmados
#dfresumen5=dfresumen5ea.groupby(['Jurisdiccion','sexo','+60','clasificacion','agrupamiento_confirmados','Cod ISO 3166 alpha 2','pob_j_2021'],dropna=False).agg(
#    casos_confirmados=('id_evento_caso','count'),
#    fallecidos=('cuenta_fallecidos','sum')
#)

#Total_de_registros5=dfresumen5.sum()
#print(dfresumen5)
#print('Total de control',Total_de_registros5)
